In [ ]:
# http://www.vgchartz.com/gamedb/
# https://github.com/GregorUT/vgchartzScrape
# https://github.com/GregorUT/vgchartzScrape/blob/master/vgchartzfull.py

In [ ]:
from bs4 import BeautifulSoup, element
from urllib.request import urlopen
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
# https://medium.com/@harshit4084/track-your-loop-using-tqdm-7-ways-progress-bars-in-python-make-things-easier-fcbbb9233f24

In [ ]:
url_base = 'https://www.vgchartz.com/games/games.php?page='

url_tail  = 'name=&keyword=&console=&region=All&developer=&publisher=' #'&goty_year='
url_tail_b  = '&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=500'
url_tail_b += '&order=ReleaseDate&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1'
url_tail_b += '&showvgchartzscore=0&showvgchartzscore=1&shownasales=0&showdeveloper=0&showdeveloper=1'
url_tail_b += '&showcriticscore=0&showcriticscore=1&showpalsales=0&showreleasedate=0&showreleasedate=1'
url_tail_b += '&showuserscore=0&showuserscore=1&showjapansales=0&showlastupdate=0&showlastupdate=1'
url_tail_b += '&showothersales=0&showshipped=0&showshipped=1'

In [ ]:
url = url_base + str(1) + url_tail + url_tail_b
url

In [ ]:
g_name = []
g_platform = []
g_publisher = []
g_developer = []
critic_score = []
user_score = []
sales_global = []
vgs_score = []
release_date = []
genre= []

for p in tqdm(range(1,2), desc= 'Main loop vgchartz'):  #124
    url = url_base + str(p) + url_tail + url_tail_b
     #   url = url_base + str(page) + url_tail + str(year) + url_tail_b

    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    
    game_tags = list(filter( lambda x: x.attrs['href'].startswith('https://www.vgchartz.com/game/'),
                             soup.findAll(lambda tag: tag.name == 'a' and 'href' in tag.attrs)))
    
    for tag in game_tags:
        g_name.append(tag.text.strip())
        
        # 2 levels up a --> td --> tr
        data = tag.parent.parent.find_all("td")

        g_platform.append(data[3].find('img').attrs['alt'])
        g_publisher.append(data[4].string.strip())
        g_developer.append(data[5].string.strip())

        vgs_score.append(
            float(data[6].string) if not data[6].string.startswith("N/A") else np.nan)
        critic_score.append(
            float(data[7].string) if not data[7].string.startswith("N/A") else np.nan)
        user_score.append(
            float(data[8].string) if not data[8].string.startswith("N/A") else np.nan)
        sales_global.append(
            float(data[10].string[:-1]) if not data[10].string.startswith("N/A") else np.nan)
        release_date.append(
            data[11].string.strip() if data[11].string.strip() != 'N/A' else np.nan)
    
    continue 
    
    for tag in tqdm(game_tags, total=len(game_tags), desc= f'page {p}'):
        
        url_to_game = tag.attrs['href']
        site_raw = urlopen(url_to_game)
        sub_soup = BeautifulSoup(site_raw, "html.parser")
        # again, the info box is inconsistent among games so we
        # have to find all the h2 and traverse from that to the genre name
        h2s = sub_soup.find("div", {"id": "gameGenInfoBox"}).find_all('h2')
        # make a temporary tag here to search for the one that contains
        # the word "Genre"
        temp_tag = element.Tag
        for h2 in h2s:
            if h2.string == 'Genre':
                temp_tag = h2
                break
        genre.append(temp_tag.next_sibling.string)


vgchartz = pd.DataFrame({'title': g_name,
              'platform': g_platform,
              'publisher': g_publisher,
              'developer': g_developer,
              'critic_score': critic_score,
              'user_score': user_score,
              'sales_global': sales_global,
              'vgs_score': vgs_score,
              'release date' : release_date,
              #'genre': genre
             })

vgchartz.head()

In [ ]:
vgchartz.info()

In [ ]:
vgchartz[(vgchartz['sales_global']>0)&(vgchartz['sales_global']<7)]

In [ ]:
vgchartz.to_csv("vgsales_vgchartz_v1_no_genre.csv", sep=",", encoding='utf-8', index=False)

In [ ]:
from  time import sleep
sleep(1)

In [ ]:
df = pd.DataFrame()

for i in range(5):
    df = df.append({'A': i}, ignore_index=True)
df = df.append({'B': 5}, ignore_index=True)
df = df.append({'A': -1,'B': 5}, ignore_index=True)    

df.head(10)
    
